# Character Recurrent Neural Network
- Abstract
- Long short-term memory(LSTM)


## 1. Settings
### 1) Import required libraries

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [54]:
import unidecode
import string
import random
import re
import time, math
from utils2 import *

## 2) Hyperparameter

In [152]:
num_epochs = 5000
hidden_size = 100
output_size=1
batch_size =1
num_layers = 3
lr = 0.002
NUM_STEPS = 500
chunk_len=200
path = './data/bitstamp.csv'

In [56]:
# train Parameters
seq_length = 200
sub_seq_length = 150
data_dim = 18
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 101
plot_name = 'LSTM_new_01'

## 2. Data

### 2) Get data

In [59]:
data = data_loading(path)

In [60]:
len(data)

2171

### 3) Chunk into input & label

In [61]:
def random_chunk():
    start_index = random.randint(0, len(data) - chunk_len)
    end_index = start_index + chunk_len + 1
    return data[start_index:end_index]

print(random_chunk().shape)

(201, 18)


In [144]:
def random_training_set():    
    chunk = random_chunk()
    inp = torch.from_numpy(chunk[:-1])
    target = torch.from_numpy(chunk[1:,-1])
    return Variable(inp).cuda(), Variable(target).cuda()


## 3. Model & Optimizer
### 1) Model

In [112]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.encoder = nn.Linear(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    def forward(self, input, hidden,cell):
        out = self.encoder(input.view(1,-1))
        #out = self.encoder(input)
        out,(hidden,cell) = self.rnn(out.view(1,1,-1),(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        return out,hidden,cell
    def init_hidden(self):
        hidden = Variable(torch.zeros(num_layers,batch_size,hidden_size)).cuda()
        cell = Variable(torch.zeros(num_layers,batch_size,hidden_size)).cuda()
        return hidden,cell
model = RNN(data.shape[1], hidden_size, output_size, num_layers).cuda()

### 2) Loss & Optimizer

In [102]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.MSELoss()

### 3) Training

In [ ]:
for i in range(num_epochs):
    inp , label = random_training_set()
    hidden,cell = model.init_hidden()
    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j]
        y,hidden,cell = model(x,hidden,cell)
        loss += loss_func(y,y_)
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")


 Variable containing:
 14.4040
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 36.7881
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 33.5993
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 29.9828
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 34.0142
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 37.3208
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 3.0452
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 4.4625
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 2.8919
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 41.0649
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 38.4514
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 2.1919
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 


 Variable containing:
 44.7611
[torch.cuda.FloatTensor of size 1 (G